In [2]:
# breakhistf2v026V201-ncvs01.ipynb  v1.0.0 
# 
#  --------------------------------------------------
#  Hangzhou Domain Zones Technology Co., Ltd

#  Apache Licence 2.0       https://www.apache.org/licenses/LICENSE-2.0
#  --------------------------------------------------


import os
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow import keras
import neural_structured_learning as nsl
from tensorflow.python.keras.api._v2.keras import layers, optimizers, losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')
  
%matplotlib inline



def read_csv(csvnamepath, filename, is2=False):
     

    # read from csv file
    images, labels = [], []
    benigns = ["adenosis" ,"fibroadenoma" ,"tubular_adenoma" , "phyllodes_tumor"]
    with open(os.path.join(csvnamepath, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label, cancername = row
            if is2:
                if  cancername in benigns:
                    label = 0
                else:
                    label = 1
            else:
                label = int(label)

            images.append(img)
            labels.append(label)

    assert len(images) == len(labels)

    return images, labels

# @tf.function 
def load_breakhis(dirc, filename, is2 = False , mode='train' ):
    classdir2label={}
     
    filedirs = os.listdir( dirc)
    for filedir in filedirs:
        if not os.path.isdir(os.path.join(dirc,filedir)):
            continue
        classdir2label[filedir]=len(classdir2label.keys())
    

     
    images, labels = read_csv(os.path.join(os.path.abspath('.'),'tf2breakhis'), filename ,is2)  
    if mode == 'train':  # 70%
        images = images[:int(0.7 * len(images))]
        labels = labels[:int(0.7 * len(labels))]
    elif mode == 'val':  # 10% = 70%->80%
        images = images[int(0.7 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.7 * len(labels)):int(0.8 * len(labels))]
    else:  # 20% = 80%->100%
        images = images[int(0.8 * len(images)):]
        labels = labels[int(0.8 * len(labels)):]
    return images, labels, classdir2label


  
img_mean = tf.constant([0.485, 0.456, 0.406])
img_std = tf.constant([0.229, 0.224, 0.225])

@tf.function
def normalize(x, mean=img_mean, std=img_std):
    # x shape: [224, 224, 3]
    # mean：shape为1；这里用到了广播机制。我们安装好右边对齐的原则，可以得到如下；
    # mean : [1, 1, 3], std: [3]        先插入1
    # mean : [224, 224, 3], std: [3]    再变为224
    x = (x - mean)/std
    return x

# 数据normalize之后，这里有一个反normalizaion操作。比如数据可视化的时候，需要反过来。
@tf.function
def denormalize(x, mean=img_mean, std=img_std):
    x = x * std + mean
    return x

 


# 预处理的函数，复制过来。
@tf.function
def preprocess(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
   
    x = tf.image.random_flip_left_right(x)
    #plt.imshow(x)
  
    x = tf.image.random_flip_up_down(x)
    
    x = tf.image.random_crop(x, [224,224,3])
    #plt.imshow(x)
    
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y


# 预处理的函数，复制过来。
@tf.function
def preprocess1(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
    
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
      
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y




 

batchsz = 32
epochnum =10
# creat train db   一般训练的时候需要shuffle。其它是不需要的。

    
images, labels, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'),  "tf2breakhisnewCSV",False,'train')

#images, labels, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'train')
db_train = tf.data.Dataset.from_tensor_slices((images, labels))  # 变成个Dataset对象。
db_train = db_train.shuffle(1000).repeat(4).map(preprocess).shuffle(1000).batch(batchsz)  # map函数图片路径变为内容。
 

# crate validation db
images2, labels2, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), "tf2breakhisnewCSV",False,'val')
db_val = tf.data.Dataset.from_tensor_slices((images2, labels2))
db_val = db_val.shuffle(1000).repeat(4).map(preprocess).shuffle(1000).batch(batchsz)
# create test db
images3, labels3, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), "tf2breakhisnewCSV",False,'test')
db_test = tf.data.Dataset.from_tensor_slices((images3, labels3))
db_test = db_test.map(preprocess1).batch(batchsz)

 
resnetdense = keras.Sequential([
    layers.Conv2D(16,5,3),
    layers.MaxPool2D(3,3),
    layers.ReLU(),
    layers.Conv2D(64,5,3),
    layers.MaxPool2D(2,2),
    layers.ReLU(),
    
    layers.Flatten(),
    #
    layers.Dense(128), 
    layers.Dropout(rate=0.5),
    
    
    
    layers.Dense(64),  
    layers.ReLU(),
    layers.Dense(8)
   ])


 

# 首先创建Resnet18
# resnet = ResNet(8)

resnetdense.build(input_shape=(batchsz, 224, 224, 3))
resnetdense.summary()

# monitor监听器, 连续5个验证准确率不增加，这个事情触发。
 
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=100

)

  

 
# 网络的装配。
resnetdense.compile(optimizer=optimizers.Adam(lr=1e-4),
               loss=losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

# 完成标准的train，val, test;
# 标准的逻辑必须通过db_val挑选模型的参数，就需要提供一个earlystopping技术，

LOGDIR='log/breakhistf2v026' 
resnetdense.fit(db_train, validation_data=db_val, 
                        validation_freq=10, epochs=1000, 
                        callbacks=[TensorBoard(log_dir=LOGDIR)])   # 1个epoch验证1次。触发了这个事情，提前停止了。
resnetdense.evaluate(db_test)
 
 

    
    
 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  25664     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0

692/692 [==============================] - 532s 769ms/step - loss: 0.4044 - accuracy: 0.8415 - val_loss: 0.5592 - val_accuracy: 0.7829
Epoch 131/1000
692/692 [==============================] - 473s 684ms/step - loss: 0.4084 - accuracy: 0.8399
Epoch 132/1000
692/692 [==============================] - 470s 679ms/step - loss: 0.4067 - accuracy: 0.8405
Epoch 133/1000
692/692 [==============================] - 477s 689ms/step - loss: 0.4014 - accuracy: 0.8398
Epoch 134/1000
692/692 [==============================] - 476s 688ms/step - loss: 0.3942 - accuracy: 0.8442
Epoch 135/1000
692/692 [==============================] - 473s 684ms/step - loss: 0.3948 - accuracy: 0.8429
Epoch 136/1000
692/692 [==============================] - 478s 691ms/step - loss: 0.3943 - accuracy: 0.8425
Epoch 137/1000
692/692 [==============================] - 477s 689ms/step - loss: 0.3926 - accuracy: 0.8436
Epoch 138/1000
692/692 [==============================] - 476s 688ms/step - loss: 0.3864 - accuracy: 0.8470
E

692/692 [==============================] - 476s 688ms/step - loss: 0.2707 - accuracy: 0.8934
Epoch 276/1000
692/692 [==============================] - 478s 691ms/step - loss: 0.2743 - accuracy: 0.8930
Epoch 277/1000
692/692 [==============================] - 473s 683ms/step - loss: 0.2704 - accuracy: 0.8920
Epoch 278/1000
692/692 [==============================] - 479s 693ms/step - loss: 0.2665 - accuracy: 0.8933
Epoch 279/1000
692/692 [==============================] - 477s 690ms/step - loss: 0.2644 - accuracy: 0.8909
Epoch 280/1000
692/692 [==============================] - 536s 774ms/step - loss: 0.2647 - accuracy: 0.8946 - val_loss: 0.5702 - val_accuracy: 0.7924
Epoch 281/1000
692/692 [==============================] - 475s 687ms/step - loss: 0.2655 - accuracy: 0.8926
Epoch 282/1000
692/692 [==============================] - 473s 683ms/step - loss: 0.2700 - accuracy: 0.8887
Epoch 283/1000
692/692 [==============================] - 478s 690ms/step - loss: 0.2632 - accuracy: 0.8963
E

Epoch 421/1000
692/692 [==============================] - 479s 693ms/step - loss: 0.2130 - accuracy: 0.9148
Epoch 422/1000
692/692 [==============================] - 474s 685ms/step - loss: 0.2091 - accuracy: 0.9148
Epoch 423/1000
692/692 [==============================] - 477s 690ms/step - loss: 0.2109 - accuracy: 0.9156
Epoch 424/1000
692/692 [==============================] - 477s 689ms/step - loss: 0.2115 - accuracy: 0.9146
Epoch 425/1000
692/692 [==============================] - 474s 686ms/step - loss: 0.2160 - accuracy: 0.9113
Epoch 426/1000
692/692 [==============================] - 474s 685ms/step - loss: 0.2136 - accuracy: 0.9148
Epoch 427/1000
692/692 [==============================] - 476s 687ms/step - loss: 0.2141 - accuracy: 0.9138
Epoch 428/1000
692/692 [==============================] - 477s 689ms/step - loss: 0.2087 - accuracy: 0.9173
Epoch 429/1000
692/692 [==============================] - 477s 689ms/step - loss: 0.2137 - accuracy: 0.9142
Epoch 430/1000
692/692 [====

692/692 [==============================] - 444s 642ms/step - loss: 0.1949 - accuracy: 0.9219
Epoch 495/1000
692/692 [==============================] - 444s 641ms/step - loss: 0.1885 - accuracy: 0.9245
Epoch 496/1000
692/692 [==============================] - 441s 638ms/step - loss: 0.1958 - accuracy: 0.9209
Epoch 497/1000
692/692 [==============================] - 442s 638ms/step - loss: 0.1925 - accuracy: 0.9206
Epoch 498/1000
692/692 [==============================] - 449s 649ms/step - loss: 0.1947 - accuracy: 0.9229
Epoch 499/1000
692/692 [==============================] - 446s 644ms/step - loss: 0.1949 - accuracy: 0.9195
Epoch 500/1000
692/692 [==============================] - 507s 732ms/step - loss: 0.1938 - accuracy: 0.9229 - val_loss: 0.4885 - val_accuracy: 0.8372
Epoch 501/1000
692/692 [==============================] - 441s 638ms/step - loss: 0.1929 - accuracy: 0.9211
Epoch 502/1000
692/692 [==============================] - 442s 639ms/step - loss: 0.1886 - accuracy: 0.9219
E

692/692 [==============================] - 536s 774ms/step - loss: 0.1674 - accuracy: 0.9319 - val_loss: 0.5500 - val_accuracy: 0.8369
Epoch 641/1000
692/692 [==============================] - 476s 688ms/step - loss: 0.1793 - accuracy: 0.9267
Epoch 642/1000
692/692 [==============================] - 480s 694ms/step - loss: 0.1718 - accuracy: 0.9297
Epoch 643/1000
692/692 [==============================] - 479s 693ms/step - loss: 0.1719 - accuracy: 0.9298
Epoch 644/1000
692/692 [==============================] - 476s 688ms/step - loss: 0.1780 - accuracy: 0.9290
Epoch 645/1000
692/692 [==============================] - 479s 692ms/step - loss: 0.1657 - accuracy: 0.9339
Epoch 646/1000
692/692 [==============================] - 447s 646ms/step - loss: 0.1725 - accuracy: 0.9296
Epoch 647/1000
692/692 [==============================] - 452s 654ms/step - loss: 0.1692 - accuracy: 0.9324
Epoch 648/1000
692/692 [==============================] - 447s 647ms/step - loss: 0.1694 - accuracy: 0.9307
E

692/692 [==============================] - 475s 687ms/step - loss: 0.1616 - accuracy: 0.9350
Epoch 786/1000
692/692 [==============================] - 475s 686ms/step - loss: 0.1578 - accuracy: 0.9367
Epoch 787/1000
692/692 [==============================] - 474s 685ms/step - loss: 0.1557 - accuracy: 0.9367
Epoch 788/1000
692/692 [==============================] - 471s 681ms/step - loss: 0.1546 - accuracy: 0.9371
Epoch 789/1000
692/692 [==============================] - 475s 687ms/step - loss: 0.1529 - accuracy: 0.9384
Epoch 790/1000
692/692 [==============================] - 516s 745ms/step - loss: 0.1550 - accuracy: 0.9366 - val_loss: 0.5766 - val_accuracy: 0.8347
Epoch 791/1000
692/692 [==============================] - 445s 642ms/step - loss: 0.1575 - accuracy: 0.9347
Epoch 792/1000
692/692 [==============================] - 439s 635ms/step - loss: 0.1531 - accuracy: 0.9361
Epoch 793/1000
692/692 [==============================] - 456s 659ms/step - loss: 0.1561 - accuracy: 0.9371
E

Epoch 931/1000
692/692 [==============================] - 443s 641ms/step - loss: 0.1418 - accuracy: 0.9436
Epoch 932/1000
692/692 [==============================] - 497s 718ms/step - loss: 0.1402 - accuracy: 0.9439
Epoch 933/1000
692/692 [==============================] - 471s 681ms/step - loss: 0.1450 - accuracy: 0.9407
Epoch 934/1000
692/692 [==============================] - 476s 688ms/step - loss: 0.1423 - accuracy: 0.9412
Epoch 935/1000
692/692 [==============================] - 473s 684ms/step - loss: 0.1450 - accuracy: 0.9419
Epoch 936/1000
692/692 [==============================] - 473s 683ms/step - loss: 0.1395 - accuracy: 0.9428
Epoch 937/1000
692/692 [==============================] - 476s 688ms/step - loss: 0.1405 - accuracy: 0.9415
Epoch 938/1000
692/692 [==============================] - 474s 685ms/step - loss: 0.1484 - accuracy: 0.9390 - loss: 0
Epoch 939/1000
692/692 [==============================] - 473s 684ms/step - loss: 0.1439 - accuracy: 0.9419
Epoch 940/1000
692

[0.4829020789265633, 0.863464]

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            multiple                  1216      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 multiple                  0         
_________________________________________________________________
re_lu_3 (ReLU)               multiple                  0         
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  25664     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 multiple                  0         
_________________________________________________________________
re_lu_4 (ReLU)               multiple                  0         
_________________________________________________________________
flatten_1 (Flatten)          multiple                 

297/297 [==============================] - 263s 887ms/step - loss: 0.5316 - accuracy: 0.7963 - val_loss: 0.7044 - val_accuracy: 0.7336
Epoch 131/1000
297/297 [==============================] - 201s 678ms/step - loss: 0.5327 - accuracy: 0.7941
Epoch 132/1000
297/297 [==============================] - 201s 676ms/step - loss: 0.5265 - accuracy: 0.7983
Epoch 133/1000
297/297 [==============================] - 201s 678ms/step - loss: 0.5158 - accuracy: 0.7994
Epoch 134/1000
297/297 [==============================] - 202s 679ms/step - loss: 0.5240 - accuracy: 0.8008
Epoch 135/1000
297/297 [==============================] - 202s 681ms/step - loss: 0.5296 - accuracy: 0.7982
Epoch 136/1000
297/297 [==============================] - 201s 677ms/step - loss: 0.5164 - accuracy: 0.8008
Epoch 137/1000
297/297 [==============================] - 204s 687ms/step - loss: 0.5173 - accuracy: 0.8000
Epoch 138/1000
297/297 [==============================] - 201s 678ms/step - loss: 0.5102 - accuracy: 0.8013
E

297/297 [==============================] - 187s 630ms/step - loss: 0.3479 - accuracy: 0.8636
Epoch 276/1000
297/297 [==============================] - 187s 631ms/step - loss: 0.3348 - accuracy: 0.8674
Epoch 277/1000
297/297 [==============================] - 188s 633ms/step - loss: 0.3428 - accuracy: 0.8641
Epoch 278/1000
297/297 [==============================] - 187s 628ms/step - loss: 0.3304 - accuracy: 0.8663
Epoch 279/1000
297/297 [==============================] - 186s 628ms/step - loss: 0.3517 - accuracy: 0.8609
Epoch 280/1000
297/297 [==============================] - 244s 820ms/step - loss: 0.3324 - accuracy: 0.8690 - val_loss: 0.6410 - val_accuracy: 0.7835
Epoch 281/1000
297/297 [==============================] - 188s 632ms/step - loss: 0.3404 - accuracy: 0.8659
Epoch 282/1000
297/297 [==============================] - 186s 626ms/step - loss: 0.3341 - accuracy: 0.8655
Epoch 283/1000
297/297 [==============================] - 186s 627ms/step - loss: 0.3293 - accuracy: 0.8724
E

Epoch 421/1000
297/297 [==============================] - 187s 629ms/step - loss: 0.2694 - accuracy: 0.8972
Epoch 422/1000
297/297 [==============================] - 189s 637ms/step - loss: 0.2466 - accuracy: 0.9016
Epoch 423/1000
297/297 [==============================] - 188s 634ms/step - loss: 0.2434 - accuracy: 0.9063
Epoch 424/1000
297/297 [==============================] - 187s 630ms/step - loss: 0.2570 - accuracy: 0.8950
Epoch 425/1000
297/297 [==============================] - 187s 629ms/step - loss: 0.2668 - accuracy: 0.8955
Epoch 426/1000
297/297 [==============================] - 187s 629ms/step - loss: 0.2557 - accuracy: 0.8967
Epoch 427/1000
297/297 [==============================] - 187s 629ms/step - loss: 0.2411 - accuracy: 0.9014
Epoch 428/1000
297/297 [==============================] - 187s 629ms/step - loss: 0.2599 - accuracy: 0.8964
Epoch 429/1000
297/297 [==============================] - 188s 632ms/step - loss: 0.2428 - accuracy: 0.9017
Epoch 430/1000
297/297 [====

297/297 [==============================] - 202s 680ms/step - loss: 0.2212 - accuracy: 0.9131
Epoch 495/1000
297/297 [==============================] - 202s 678ms/step - loss: 0.2164 - accuracy: 0.9134
Epoch 496/1000
297/297 [==============================] - 203s 683ms/step - loss: 0.2338 - accuracy: 0.9072
Epoch 497/1000
297/297 [==============================] - 202s 681ms/step - loss: 0.2235 - accuracy: 0.9120
Epoch 498/1000
297/297 [==============================] - 204s 685ms/step - loss: 0.2191 - accuracy: 0.9103
Epoch 499/1000
297/297 [==============================] - 204s 687ms/step - loss: 0.2143 - accuracy: 0.9171
Epoch 500/1000
297/297 [==============================] - 264s 888ms/step - loss: 0.2279 - accuracy: 0.9115 - val_loss: 0.6929 - val_accuracy: 0.7924
Epoch 501/1000
297/297 [==============================] - 203s 683ms/step - loss: 0.2125 - accuracy: 0.9138
Epoch 502/1000
297/297 [==============================] - 203s 682ms/step - loss: 0.2151 - accuracy: 0.9159
E

297/297 [==============================] - 262s 884ms/step - loss: 0.1972 - accuracy: 0.9233 - val_loss: 0.7675 - val_accuracy: 0.7882
Epoch 641/1000
297/297 [==============================] - 200s 674ms/step - loss: 0.1982 - accuracy: 0.9236
Epoch 642/1000
297/297 [==============================] - 203s 685ms/step - loss: 0.1779 - accuracy: 0.9326
Epoch 643/1000
297/297 [==============================] - 202s 682ms/step - loss: 0.1879 - accuracy: 0.9246
Epoch 644/1000
297/297 [==============================] - 203s 683ms/step - loss: 0.1889 - accuracy: 0.9240
Epoch 645/1000
297/297 [==============================] - 202s 681ms/step - loss: 0.1923 - accuracy: 0.9219
Epoch 646/1000
297/297 [==============================] - 201s 677ms/step - loss: 0.1974 - accuracy: 0.9201
Epoch 647/1000
297/297 [==============================] - 205s 691ms/step - loss: 0.1879 - accuracy: 0.9221
Epoch 648/1000
297/297 [==============================] - 204s 688ms/step - loss: 0.1862 - accuracy: 0.9270
E

297/297 [==============================] - 188s 634ms/step - loss: 0.1562 - accuracy: 0.9374
Epoch 786/1000
297/297 [==============================] - 186s 626ms/step - loss: 0.1635 - accuracy: 0.9355
Epoch 787/1000
297/297 [==============================] - 188s 635ms/step - loss: 0.1620 - accuracy: 0.9368
Epoch 788/1000
297/297 [==============================] - 190s 639ms/step - loss: 0.1549 - accuracy: 0.9364
Epoch 789/1000
297/297 [==============================] - 187s 630ms/step - loss: 0.1618 - accuracy: 0.9356
Epoch 790/1000
297/297 [==============================] - 245s 827ms/step - loss: 0.1563 - accuracy: 0.9370 - val_loss: 0.7973 - val_accuracy: 0.7876
Epoch 791/1000
297/297 [==============================] - 187s 628ms/step - loss: 0.1541 - accuracy: 0.9389
Epoch 792/1000
297/297 [==============================] - 188s 632ms/step - loss: 0.1636 - accuracy: 0.9341
Epoch 793/1000
297/297 [==============================] - 189s 635ms/step - loss: 0.1633 - accuracy: 0.9323
E

Epoch 931/1000
297/297 [==============================] - 203s 684ms/step - loss: 0.1387 - accuracy: 0.9466
Epoch 932/1000
297/297 [==============================] - 202s 680ms/step - loss: 0.1421 - accuracy: 0.9445
Epoch 933/1000
297/297 [==============================] - 203s 683ms/step - loss: 0.1541 - accuracy: 0.9397
Epoch 934/1000
297/297 [==============================] - 202s 679ms/step - loss: 0.1447 - accuracy: 0.9428
Epoch 935/1000
297/297 [==============================] - 201s 678ms/step - loss: 0.1341 - accuracy: 0.9456
Epoch 936/1000
297/297 [==============================] - 203s 685ms/step - loss: 0.1389 - accuracy: 0.9451
Epoch 937/1000
297/297 [==============================] - 203s 684ms/step - loss: 0.1300 - accuracy: 0.9489
Epoch 938/1000
297/297 [==============================] - 204s 685ms/step - loss: 0.1406 - accuracy: 0.9438
Epoch 939/1000
297/297 [==============================] - 201s 676ms/step - loss: 0.1473 - accuracy: 0.9426
Epoch 940/1000
297/297 [====

[0.7688439130783081, 0.82300884]